In [1]:
import sys
import os

notebook_path = os.path.abspath(os.getcwd())
parent_dir = os.path.abspath(os.path.join(notebook_path, '..'))
sys.path.append(parent_dir)

# Создаю новый датасет очищая от:
- Повторяющихся дат $dttm = \text{01-01-2000}, price = 3; dttm = \text{01-01-2000}, price = 2$
- Арбитражных опционов $max(S_0 − K, 0) < EC < S_0 \right_arrow False, max(K − S_0, 0) < EP < K \rightarrow False$
- Нулевых цен $EC_{real} \text{ or } EP_{real} = 0$

In [2]:
import pandas as pd
from proj_utils.refactoring import find_arbitrage

data = pd.read_csv('../data/server_data/dataset.csv', index_col='Unnamed: 0')
data['S_prev'] = data['S'].shift(1)
data = data.iloc[1:]
data['price'] *= data['S']
data = data[data.apply(find_arbitrage, axis=1) == 0]
data = data.sort_values(by=['id', 'dttm'])

date_prev = None
option_id_prev = None

bad_index = []

for idx, row in data.iterrows():
    
    if row.price == 0:
        bad_index.append(idx)
        continue

    if date_prev is None:
        date_prev = row.dttm
        option_id_prev = row.id
        continue

    if option_id_prev != row.id:
        option_id_prev = row.id
        date_prev = row.dttm
        continue

    if date_prev == row.dttm and option_id_prev == row.id:
        date_prev = row.dttm
        bad_index.append(idx)

data.drop(bad_index, inplace=True)

data.to_csv('../data/server_data/dataset_cleaned.csv')